In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os
import gmaps
import run_heatmap_tools as rht


In [3]:
import getpass
gmap_token = getpass.getpass('Enter your Google Maps API token:')

Enter your Google Maps API token:········


In [4]:
gmaps.configure(api_key=gmap_token)

In [5]:
folder = os.getcwd()+'\\run_logs'

In [6]:
# Read all run data, combine into lat & long dataframe
lat = []
long = []

for entry in os.scandir(folder):
    if entry.path.endswith(".tcx"):
       # print(entry.path) # Keeping this here to QC which files are being considered
        with open(entry.path,'r',encoding="utf-8") as myFile:
            results = rht.parser(myFile, {'{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LatitudeDegrees', '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LongitudeDegrees'})
            for tag, text in results:
                #print(tag, text)
                if tag == '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LatitudeDegrees':
                    text = float(text)
                    lat.append(text)
                if tag == '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LongitudeDegrees':
                    text = float(text)
                    long.append(text)
        myFile.close()
            
data = [lat,long]
run_df = pd.DataFrame(data)


run_df = run_df.transpose()
run_df.columns = 'Latitude Longitude'.split()

In [7]:
# make figure
fig = gmaps.figure(map_type='HYBRID')
r=run_df[['Latitude','Longitude']]

t_layer = gmaps.heatmap_layer(r)
t_layer.max_intensity = 200
t_layer.point_radius = 7

fig.add_layer(t_layer)
fig

Figure(layout=FigureLayout(height='420px'))